In [29]:
using Globtim
using CSV
using DataFrames
include("../src/lib_func.jl")

# Load the dataframe from the CSV file
df_2d = CSV.read("../data/camel_3_d6.csv", DataFrame)

# Constants and Parameters
d = 8      # Degree 
const n, a, b = 4, 5, 1 
const scale_factor = a / b  # Scaling constant, C is appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = .1 , 2 / 10  # Sampling parameters
const tol_l2 = 1e-1               # Define the tolerance for the L2-norm
# The objective function
f = camel_3_by_3 # Objective function

camel_3_by_3 (generic function with 1 method)

In [30]:
while true # Potential infinite loop
    global poly_approx = MainGenerate(f, n, d, delta, alpha, scale_factor, 0.2) # computes the approximant in Chebyshev basis
    if poly_approx.nrm < tol_l2
        println("attained the desired L2-norm: ", poly_approx.nrm)
        break
    else
        println("current L2-norm: ", poly_approx.nrm)
        println("Number of samples: ", poly_approx.N)
        global d += 1
    end
end

attained the desired L2-norm: 1.9420523675820383e-14


In [31]:
using DynamicPolynomials, HomotopyContinuation, ProgressLogging, DataFrames
@polyvar(x[1:n]) # Define polynomial ring 
ap = main_nd(n, d, poly_approx.coeffs, x)
# Expand the polynomial approximant to the standard monomial basis in the Lexicographic order w.r.t x. 
PolynomialApproximant = sum(Float64.(ap) .* MonomialVector(x, 0:d))
grad = differentiate.(PolynomialApproximant, x)
sys = System(grad)

Check


System of length 4
 4 variables: x₁, x₂, x₃, x₄

 2.43415319575487e-13 + 99.9999999999769*x₁ + 25.0000000000003*x₂ - 7.24064074740531e-14*x₃ - 5.1827713070706e-14*x₄ - 1.32895281777051e-12*x₂*x₁ - 3.31777682684502e-13*x₂*x₁^2 - 2.44788139820558e-12*x₂*x₁^3 - 2.59990517187531e-12*x₂*x₁^4 + 3.35464885541138e-12*x₂*x₁^5 + 3.22295740034157e-12*x₂*x₁^6 - 7.20456906370585e-14*x₂*x₃ - 1.5695541745624e-13*x₂*x₃^2 + 1.50300526483121e-13*x₂*x₃^3 + 5.60008498814279e-13*x₂*x₃^4 - 7.3176841864824e-14*x₂*x₃^5 - 3.61859120764922e-13*x₂*x₃^6 - 1.25768053137526e-11*x₂^2*x₁ - 1.50141189480057e-13*x₂^2*x₁^2 - 3.77720878237962e-12*x₂^2*x₁^3 + 1.89497621617569e-12*x₂^2*x₁^4 - 1.31903914250445e-11*x₂^2*x₁^5 - 3.09895121209936e-13*x₂^2*x₃ - 2.26198542291971e-13*x₂^2*x₃^2 + 2.44422866349608e-13*x₂^2*x₃^3 + 1.92498676432205e-13*x₂^2*x₃^4 + 2.7695997566876e-13*x₂^2*x₃^5 + 3.26494164298552e-12*x₂^3*x₁ + 4.44087297588376e-12*x₂^3*x₁^2 + 9.47817933620604e-13*x₂^3*x₁^3 - 1.5640881499713e-12*x₂^3*x₁^4 + 1.8692256518

Now we use Optim.jl package to initialize step 3 and converge to the local minimizers.
How do we know we have everything ? Use simple combinations of 

In [32]:
Real_sol_lstsq = HomotopyContinuation.solve(sys)
real_pts = HomotopyContinuation.real_solutions(Real_sol_lstsq; only_real=true, multiple_results=false)

Tracking 2401 paths...   0%|                            |  ETA: 22:11:52



Tracking 2401 paths...   0%|▏                           |  ETA: 4:26:23



Tracking 2401 paths...   1%|▎                           |  ETA: 2:36:45



Tracking 2401 paths...   1%|▎                           |  ETA: 1:46:35



Tracking 2401 paths...   1%|▍                           |  ETA: 1:20:43



Tracking 2401 paths...   2%|▌                           |  ETA: 1:04:57



Tracking 2401 paths...   2%|▋                           |  ETA: 0:54:21



Tracking 2401 paths...   2%|▋                           |  ETA: 0:47:34



Tracking 2401 paths...   3%|▊                           |  ETA: 0:41:37



Tracking 2401 paths...   3%|▉                           |  ETA: 0:38:04



Tracking 2401 paths...   3%|▉                           |  ETA: 0:34:37



Tracking 2401 paths...   4%|█                           |  ETA: 0:31:20



Tracking 2401 paths...   4%|█▏                          |  ETA: 0:28:00



Tracking 2401 paths...  

57-element Vector{Vector{Float64}}:
 [1.393207381955118e7, 1.8422567277231854e6, 4.485825750789259e6, -199698.02779567632]
 [1.8245109955993246e7, 745176.8794666433, -1.71436514743506e6, 1.4321741756711323e7]
 [0.7307852319980613, -94.43780351436756, -0.34789039698569746, -1.5273105889356662]
 [-1.8245112914740227e7, -745177.9364214294, 1.7143654189439889e6, -1.4321744943210311e7]
 [0.7307820719801588, -94.43647012741319, 0.3857677999903718, -1.8826017771993373]
 [0.3495104691660765, -0.17475523458308323, 0.3495104691658264, -0.17475523458213668]
 [-0.3495104691660793, 0.17475523458314632, 0.34951046916582496, -0.17475523458213707]
 [0.21410845836469325, -0.1070542291823565, 0.21410845836545658, -0.10705422918221502]
 [0.359657726694407, -0.6028528492561649, 0.6787245460642822, -65.84809079490002]
 [0.7481801481678804, -117.24767528789283, -0.6793316488425006, 91.14003208801982]
 ⋮
 [-0.3495104691660866, 0.17475523458315906, -0.21410845836553177, 0.10705422918204922]
 [0.34951046916608

Construct a Dataframe with only the critical points of the approximant $w_d$ which fall into the $[-1, 1]^4$ domain and rescale them to the original domain. 

In [33]:
condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1 && -1 < point[3] < 1 && -1 < point[4] < 1 
filtered_points = filter(condition, real_pts) # Filter points using the filter function
# Colllect the critical points of the approximant 
h_x1 = scale_factor * Float64[point[1] for point in filtered_points] # Initialize the x vector for critical points of approximant
h_x2 = scale_factor * Float64[point[2] for point in filtered_points] 
h_x3 = scale_factor * Float64[point[3] for point in filtered_points] 
h_x4 = scale_factor * Float64[point[4] for point in filtered_points] 

height = map(p -> f(p), zip(h_x1, h_x2, h_x3, h_x4)) # Compute the height of the critical points
df = DataFrame(x1 = h_x1, x2 = h_x2, x3 = h_x3, x4 = h_x4, height = height); # Create a DataFrame

Now we want to compute the distance between the critical points of the 2d function.

The true "exact" critical points of the 2d `Camel3` function are stored in `df_2d`. 

We compute which of those are the closest to the `[1:2]` and `[3:4]` coordinates of the critical points of our 4d approximant.  

The true critical points are stored in `df_2d`. Need to split to distance to minima first coordinate and second set of coordinates ten combine them.


In [34]:
total_distance = Float64[]
for i in 1:nrow(df)
    distances_1 = [norm([df.x1[i], df.x2[i]] - [df_2d.x[j], df_2d.y[j]]) for j in 1:nrow(df_2d)]
    distances_2 = [norm([df.x3[i], df.x4[i]] - [df_2d.x[j], df_2d.y[j]]) for j in 1:nrow(df_2d)]
    # Compute distances to all points in df_2d
    min_value_1, min_index_1 = findmin(distances_1)
    min_value_2, min_index_2 = findmin(distances_2) 

    # Extract the corresponding points
    point_1 = [df.x1[i], df.x2[i], df.x3[i], df.x4[i]]
    point_2 = [df_2d.x[min_index_1], df_2d.y[min_index_1], df_2d.x[min_index_2], df_2d.y[min_index_2]]

    # Compute total distance
    tot_dist = norm(point_1 - point_2)
    push!(total_distance, tot_dist)

end
df.total_distance = total_distance
sorted_df = sort(df, :total_distance)


Row,x1,x2,x3,x4,height,total_distance
,Float64,Float64,Float64,Float64,Float64,Float64
1,-8.64601e-14,2.97157e-13,-5.84073e-14,-5.98589e-13,4.77655e-25,6.61228e-13
2,-4.79172e-14,1.522e-13,-1.74755,0.873776,0.298638,0.0892601
3,-6.75214e-14,2.37627e-13,1.74755,-0.873776,0.298638,0.0892601
4,1.74755,-0.873776,-7.42508e-14,-4.80603e-13,0.298638,0.0892601
5,-1.74755,0.873776,-8.46337e-14,-5.80665e-13,0.298638,0.0892601
6,1.74755,-0.873776,-1.74755,0.873776,0.597277,0.126233
7,-1.74755,0.873776,-1.74755,0.873776,0.597277,0.126233
8,1.74755,-0.873776,1.74755,-0.873776,0.597277,0.126233
9,-1.74755,0.873776,1.74755,-0.873776,0.597277,0.126233


In [35]:
using Optim
df.steps = zeros(nrow(df))
df.converged = zeros(nrow(df))

for i in 1:nrow(df)
    # println("Optimizing for point $i")
    x0 = [df.x1[i], df.x2[i], df.x3[i], df.x4[i]]
    #  + 0.2 * randn(Float64, 4)
    res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=false))
    minimizer = Optim.minimizer(res)
    min_value = Optim.minimum(res)
    steps = res.iterations
    converged = Optim.converged(res)
    distance = norm(x0 - minimizer)
    # df.dist_to_loc_min[i] = distance
    df.steps[i] = steps
    df.converged[i] = converged
    # summary(res)
end
df

Row,x1,x2,x3,x4,height,total_distance,steps,converged
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.74755,-0.873776,1.74755,-0.873776,0.597277,0.126233,0.0,1.0
2,-1.74755,0.873776,1.74755,-0.873776,0.597277,0.126233,0.0,1.0
3,1.07054,-0.535271,1.07054,-0.535271,1.75472,0.334013,0.0,1.0
4,-1.74755,0.873776,-8.46337e-14,-5.80665e-13,0.298638,0.0892601,0.0,1.0
5,-1.74755,0.873776,-1.74755,0.873776,0.597277,0.126233,0.0,1.0
6,-6.75214e-14,2.37627e-13,1.74755,-0.873776,0.298638,0.0892601,0.0,1.0
7,-1.74755,0.873776,1.07054,-0.535271,1.176,0.252487,0.0,1.0
8,-7.52108e-14,2.59562e-13,1.07054,-0.535271,0.877362,0.236183,0.0,1.0
9,1.74755,-0.873776,-1.74755,0.873776,0.597277,0.126233,0.0,1.0
